In [15]:
import numpy as np
import random
import math
from scipy import integrate

In [5]:
# TODO: bug at E_x(0,30)
def E_x(l,d):
    """
    returns E(X) given l likes and d dislikes
    """
    
    def pdf(x):
        """
        underlying pdf for the product,P(X = q) 
        """
        if x > 0.4 and x < 0.6:
            return 5
        else:
            return 0
        
    def e_x(q):
        """
        E(x^l(1-x)^q)
        """
        return q**l*(1 - q)**d * pdf(q)

    def p_x(q):
        return q**(l+1)*(1 - q)**d * pdf(q)
    
    def integration():
        """
        E(X_{l,d}) = int_q q P(x_{l,d} = q) dq
        """
        return integrate.quad(p_x, 0,1)[0] / integrate.quad(e_x,0,1)[0]

    return integration()

def prior(x):
    return x*0.2 + 0.4

def V_x(depth, delta = 0.9, c = 0.3):
    """
    this function computes an approximation for V(X), 
    based on calculating up to depth product reviews
    """
    recursive_v = {}
    lowest = []
    for l in range(depth + 1):
        d = depth - l
#         print(l, depth)
    #     lowest[d] = d dislikes, l likes
        v_hat = max(0, (prior(l / depth) - c)/(1 - delta)) # assume uniform x0
        lowest.append(v_hat) 

    recursive_v[depth] = lowest
    
    for curr_depth in range(depth-1, -1, -1):
        lowest = []
#         print("---", curr_depth)
        for l in range(curr_depth+1):
            d = curr_depth - l
            p_like = E_x(l, d) # (l + 1) / (curr_depth + 2) # should be E_x(l, d) # TODO? E(X_{l,d}) = int 0~1 p 
            p_disl = 1 - p_like # (d + 1) / (curr_depth + 2)
            future_value = delta * ( p_like * recursive_v[curr_depth+1][l] + p_disl * recursive_v[curr_depth+1][l+1])
#             print(l,d, p_like, p_disl, E_x(l,d), future_value)
            value = max(0, E_x(l,d) - c  + future_value)
#             print(value)
            lowest.append(value)

        recursive_v[curr_depth] = lowest

    
    return recursive_v

In [13]:
depth = 100
recursive_v = V_x(depth, 0.9, 0.45)
# for i in range(depth, -1, -1):
#     print(i, recursive_v[i])
recursive_v[0]

[0.4985722950409418]

In [44]:
for i in range(50):
    print(i, E_x(0, i))

0 0.49999999999999983
1 0.49333333333084217
2 0.4868421052583055
3 0.4806153846079073
4 0.4747235386938914
5 0.469215896870335
6 0.46412093247172437
7 0.45944840952559285
8 0.4551927389997339
9 0.4513368061939597
10 0.447855689206549
11 0.4447198918275768
12 0.44189792247052195
13 0.43935706055939033
14 0.4370706690046985
15 0.43500802327527577
16 0.4331302224040316
17 0.4314487859446004
18 0.4299129247755551
19 0.4285244086009393
20 0.4278166890201325
21 0.42662910820520233
22 0.4249847956726154
23 0.4230680076901846
24 0.4221303214572783
25 0.4242208368028372
26 0.42350552258804364
27 0.41560877464182033
28 0.41104299705819575
29 0.43667452701578546
30 0.2938403634967525
31 0.42734274960924823
32 0.4269143103183715
33 0.426515196395025
34 0.42614290488251466
35 0.42579519739600735
36 0.4254700670291075
37 0.42516570988771674
38 0.4248805005692932
39 0.4246129710038406
40 0.42436179215937436
41 0.42412575818965464
42 0.4239037726664124
43 0.42369483659325113
44 0.4234980379450038
45 0

In [53]:
def E_beta(l,d):
    return (l+1)/(l + d + 2)

dp2 = {}
def catTrapezoidWeightedInverse(l,d,c):
    # gets the number of paths to l likes, d dislikes
    f = (l,d,c)
    if f in dp2:
        return dp2[f]
    else:
#         print(f"{l} and {d} at {E_x(l,d)}")
        if E_x(l,d) - c < 0:
            dp2[f] = 0
            return dp2[f]
        elif l == 0 and d == 0:
            dp2[f] = 1
            return dp2[f]
        elif l == 0:
            dp2[f] = catTrapezoidWeightedInverse(l,d-1,c)
            return dp2[f]
        elif d == 0:
            dp2[f] = catTrapezoidWeightedInverse(l-1,d,c)
            return dp2[f]
        else:
            dp2[f] = catTrapezoidWeightedInverse(l,d-1,c) + catTrapezoidWeightedInverse(l-1,d,c)
            return dp2[f]

def prob_reach(l,d):
    prob = 1
    for i in range(l):
        prob *= E_x(i,0)
    for i in range(d):
        prob *= (1-E_x(l,i))
    return prob

In [54]:
# compute static pricing

# def nCr(n,r):
#     f = math.factorial
#     return f(n) // f(r) // f(n-r)

# def catTrapezoid(w,l,m):
#     if 0 <= l < m:
#         return nCr(w + l, l)
#     elif m <= l <= w + m -1:
#         return nCr(w+l,l) - nCr(w+l,l-m)
#     else:
#         return 0

def expected_static_profit(depth, delta = 0.9, c = 0.5, c2 = 0.45):
    if c < c2 or E_x(0,0) - c2 < 0:
        return 0
    
    tot = 0
    
    for t in range(depth):
        lowest = 0
            
        for l in range(lowest,t + 1,1):
            d = t - l
            paths = catTrapezoidWeightedInverse(l,d,c)
#             print(f"t: {t}, w: {w}, l: {l}, paths: {paths}")
            if paths > 0 or d == 0:
                tot += delta**t*(c - c2)*paths*prob_reach(l,d)
            elif paths == 0 and catTrapezoidWeightedInverse(l,d-1,c) > 0: # final loss
#                 print('got here',delta**t*(c - c2)*catTrapezoidWeightedInverse(x,w,l-1,p,q,c)*prob_reach(x,w,l-1,p,q)*lose(x_update(x,w,l-1,p,q),p,q))
                tot += delta**t*(c - c2)*catTrapezoidWeightedInverse(l,d-1,c)*prob_reach(l,d-1)*(1-E_x(l,d-1))
    return tot
    
    
    

In [57]:
expected_static_profit(100, 0.9, 0.48, 0.45)

0.2571022262888802